In [1]:
from roboflow import Roboflow
import cv2
import numpy as np

In [2]:
cd C:\project\yolo_football_video_MoT

C:\project\yolo_football_video_MoT


This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.


In [3]:
from roboflow import Roboflow
import cv2
import numpy as np

# Roboflow 객체 초기화
rf = Roboflow(api_key="7LaCX25FUT6blBXw40FE")

# 워크스페이스, 프로젝트, 버전 지정
project = rf.workspace("kshi").project("chelsea_newcastle")
version = project.version(1)

# 불러온 프로젝트 버전의 모델 저장
model = version.model

loading Roboflow workspace...
loading Roboflow project...


In [9]:
# mp4파일 읽어오기
cap=cv2.VideoCapture('./data/chelsea_newcastle_test.mp4')

In [10]:
cap.read()

(True,
 array([[[ 48,  48,  46],
         [ 48,  48,  46],
         [ 48,  48,  46],
         ...,
         [  2,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[ 60,  60,  58],
         [ 60,  60,  58],
         [ 60,  60,  58],
         ...,
         [  2,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[ 65,  68,  68],
         [ 65,  68,  68],
         [ 65,  68,  68],
         ...,
         [  2,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        ...,
 
        [[ 48, 144, 113],
         [ 48, 144, 113],
         [ 52, 144, 113],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[ 87, 136, 119],
         [ 87, 136, 119],
         [ 91, 135, 119],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,  10,   0],
         [  0,  10,   0],
         [  0,   9,   0],
         ...,
         [  0,   0,  

In [11]:
cd .\prediction\video

C:\project\yolo_football_video_MoT\prediction\video


This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.


In [12]:
# 비디오 쓰기 객체 생성
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640, 480))

In [13]:
# Hex 색상코드를 RGB색상코드로 변경
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

In [32]:
(cap.read()[True]).shape

(1080, 1920, 3)

In [15]:
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        # 프레임에서 객체 감지
        prediction = model.predict(frame, confidence=50)
        
        # 결과를 프레임에 표시
        for result in prediction:
            bottom_right = (round(result['x']-result['width']/2), round(result['y']-result['height']/2))
            top_left = (round(result['x']+result['width']/2), round(result['y']+result['height']/2))
            color = hex_to_rgb(version.colors[result['class']])
            thickness = 2
            cv2.rectangle(frame, bottom_right, top_left, color, thickness)
            cv2.putText(frame, result['class'], (round(result['x']-result['width']/2), round(result['y']-result['height']/2-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color=color, thickness=3)
        
        # 결과 프레임을 파일로 저장
        out.write(frame)
    else:
        break

# 모든 작업이 완료되면 캡처와 쓰기 객체를 해제
cap.release()
out.release()

HTTPError: 408 Client Error: Request Timeout for url: https://detect.roboflow.com/chelsea_newcastle/1?api_key=7LaCX25FUT6blBXw40FE&name=YOUR_IMAGE.jpg&overlap=30&confidence=50&stroke=1&labels=false&format=json